In [53]:
import sys,os,ast
import pandas as pd
sys.path.append(os.path.join(os.path.pardir,os.path.pardir,'tcg_slb_database','python'))
from tcg_slb.phase import SLBPhase

reference="Gconfig"
version = "slb"

csvfile = '{}_phases.csv'.format(version)
em_suffix = '_{}_em'.format(version)
csv_headers = ['endmember_names','conversion_strs','X','d','W','sites','TC0','VD','SD']
if version == 'slb21':
    csv_headers.append('W_V')
def literal_return(val):
    try:
        return ast.literal_eval(val)
    except (ValueError, SyntaxError) as e:
        return None
converter = dict([(h,literal_return) for h in csv_headers])
df = pd.read_csv(os.path.join(os.path.pardir, 'tcg_slb_database','database', 'data', csvfile),skipinitialspace=True,converters=converter)

my_results = {}
for i, row in df.iterrows():
    values_dict = row.to_dict()
    name = values_dict.pop('name').replace('-','_')
    abbrev = values_dict.pop('abbrev')
    endmember_names = values_dict.pop('endmember_names')
    endmember_names = [em+em_suffix for em in endmember_names]
    phase = SLBPhase(name, abbrev, reference, endmember_names, **values_dict)
    if version == 'slb21':
        phase.G = phase.G_2021()
    Gconfig = phase.G_config_default()
    K = len(phase.endmember_names)
    if K==1:
        continue
    valdict = {"T":1000}
    for k, em in enumerate(phase.endmember_names):
        n_str = "n{}".format(k+1)
        n_val = 1./K
        valdict[n_str] = n_val
    Gconfig = Gconfig.subs(valdict)
    result = Gconfig.subs(phase.Rsym,8.31446261815324)
    my_results[phase.abbrev] = result
print(my_results)

{'plg': -5763.14632164398, 'ol': -11526.2926432880, 'wa': -11526.2926432880, 'ri': -11526.2926432880, 'pv': -14426.6440641869, 'ppv': -14426.6440641869, 'opx': -17289.4389649319, 'cpx': -22077.8636375664, 'hpcpx': -11526.2926432880, 'ak': -14426.6440641869, 'gt': -46160.7300317256, 'mw': -5763.14632164398, 'cf': -14426.6440641869, 'sp': -66815.7748130211}


In [64]:
nico_results = {
    "plg": -5763.14632164398,
    "sp": -66815.77481302111,
    "ol": -11526.29264328796,
    "wa": -11526.29264328796,
    "ri": -11526.29264328796,
    "opx": -17289.43896493194,
    "cpx": -22077.863637566377,
    "hpcpx": -11526.29264328796,
    "ak": -14426.644064186896,
    "gtmj": -46160.73003794125,
    "pv": -14426.644064186896,
    "ppv": -14426.644064186896,
    "mw": -5763.14632164398,
    "cf": -14426.644064186898,
}

for key, val in nico_results.items():
    print(my_results[key],"=",val,"?","{:.10f}".format(my_results[key]) == "{:.10f}".format(val))


-5763.14632164398 = -5763.14632164398 ? True
-66815.7748130211 = -66815.77481302111 ? True
-11526.2926432880 = -11526.29264328796 ? True
-11526.2926432880 = -11526.29264328796 ? True
-11526.2926432880 = -11526.29264328796 ? True
-17289.4389649319 = -17289.43896493194 ? True
-22077.8636375664 = -22077.863637566377 ? True
-11526.2926432880 = -11526.29264328796 ? True
-14426.6440641869 = -14426.644064186896 ? True


KeyError: 'gtmj'